In [ ]:
NEO4j_URL = ""
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""
GITHUB_TOKEN = ""
GITHUB_BASE_URL = ""

In [3]:
!pip install -q graphiti-core langchain-openai langgraph langchain-openai langchain-core

In [4]:
import asyncio
import json
import logging
import os
import sys
import uuid
from contextlib import suppress
from datetime import datetime, timezone
from pathlib import Path
from typing import Annotated

import ipywidgets as widgets
from dotenv import load_dotenv
from IPython.display import Image, display
from typing_extensions import TypedDict

load_dotenv()

False

In [ ]:
# Note: This will clear the database
# await clear_data(client.driver)
# await client.build_indices_and_constraints()

<h1>Create client for graphiti </h1>

In [5]:
from graphiti_core.llm_client.openai_client import OpenAIClient
from openai import OpenAI

from graphiti_core import Graphiti
from graphiti_core.llm_client.config import LLMConfig
from graphiti_core.embedder.openai import OpenAIEmbedder, OpenAIEmbedderConfig
from graphiti_core.cross_encoder.openai_reranker_client import OpenAIRerankerClient

from openai import AsyncOpenAI
from graphiti_core.llm_client.openai_client import OpenAIClient
from graphiti_core.llm_client.config import LLMConfig


openai_api = AsyncOpenAI(
    api_key=GITHUB_TOKEN,
    base_url=GITHUB_BASE_URL,
)

llm_config = LLMConfig(
    api_key=GITHUB_TOKEN,
    model="gpt-4.1-nano",
    small_model="gpt-4.1-nano",
    base_url=GITHUB_BASE_URL,
)

llm_client = OpenAIClient(client=openai_api, config=llm_config)
embedder_config = OpenAIEmbedderConfig(
    api_key=GITHUB_TOKEN,
    base_url=GITHUB_BASE_URL,

    embedding_model="text-embedding-3-small",
    embedding_dim=1536,
)


client = Graphiti(
    NEO4j_URL,
    NEO4J_USERNAME,
    NEO4J_PASSWORD,

    llm_client=llm_client,
    embedder=OpenAIEmbedder(config=embedder_config),

    cross_encoder=OpenAIRerankerClient(client=llm_client, config=llm_config),
)

<h1>Test create graph base on chat message</h1>

In [ ]:
import asyncio
from datetime import datetime
# from graphiti_core import GraphitiClient
from graphiti_core.nodes import EpisodeType

# The conversation stream
conversation_turns = [
"""
Officer: Where were you yesterday evening?
John: I was at the City Mall, near the food court.
"""
,"""
Officer: What time were you there?
John: I arrived at 6:30 p.m. and left around 8:00 p.m.

Officer: What were you doing at that time?
John: I was having dinner and meeting a friend.
""",
"""
Officer: Did you go anywhere else after that?
John: Yes, I went to a coffee shop on Green Street at 8:15 p.m.

Officer: What action did you take there?
John: I sat down, drank coffee, and checked my phone until 9:30 p.m.
"""
]

async def run_realtime_interview(client , data ):

  # 1. Ingest the single turn immediately
  await client.add_episode(
      name=f"Interview_Segment_{data['index']}",
      episode_body=data['content'],
      source_description="Real-time interview chat logs",
      source=EpisodeType.message,
      reference_time=datetime.now() # Current time allows Graphiti to order events
  )


In [ ]:
id = 0
data = {
    "index" : id,
    "content" : conversation_turns[id]
}
await run_realtime_interview(client , data)

<h1>Test query</h1>

In [19]:
from graphiti_core.search.search_config_recipes import COMBINED_HYBRID_SEARCH_CROSS_ENCODER

# Gán cấu hình tìm kiếm này
search_config = COMBINED_HYBRID_SEARCH_CROSS_ENCODER.model_copy(deep=True)

# Gọi _search() với config
results = await client._search(
    query="Did John have anything with him when he was at city mall last night?",
    config=search_config,
)

# Xử lý kết quả
for node in results.nodes:
    print("Node:", node.summary)

for edge in results.edges:
    print("Edge:", edge.fact)




Node: City Mall is a location where John stated he was present near the food court yesterday evening.
Node: John was at the City Mall near the food court yesterday evening from 6:30 p.m. to 8:00 p.m., having dinner and meeting a friend.
Node: The food court is a location at the City Mall where John mentioned being present yesterday evening.
Node: John met a friend for dinner at the City Mall food court on the evening of the incident, arriving at 6:30 p.m. and leaving around 8:00 p.m.
Node: Officer questioned John about his whereabouts on a specific evening; John stated he was at the City Mall near the food court from 6:30 p.m. to 8:00 p.m., having dinner and meeting a friend.
Edge: John was at the City Mall.
Edge: The City Mall contains the food court.
Edge: John met with a friend.


<h1>Implement HyDe Rag</h1>

In [17]:
import asyncio
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


llm = ChatOpenAI(
    base_url=GITHUB_BASE_URL,
    api_key=GITHUB_TOKEN,
    model="gpt-4.1-nano",
    temperature=0
)


hyde_template = """Please write a short, plausible passage that answers the question.
It doesn't have to be factually true, but it should look like a factual statement found in a database.

Question: {question}
Hypothetical Answer:"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_chain = hyde_prompt | llm | StrOutputParser()


async def retrieve_strict_raw_edge(hypothetical_answer: str):
    print(f"🔎 Đang query Graphiti (HyDE: {hypothetical_answer}")

    from graphiti_core.search.search_config_recipes import COMBINED_HYBRID_SEARCH_CROSS_ENCODER

    search_config = COMBINED_HYBRID_SEARCH_CROSS_ENCODER.model_copy(deep=True)

    results = await client._search(
        query=hypothetical_answer,
        config=search_config,
    )

    for node in results.nodes:
        print("Node:", node.summary)

    # for edge in results.edges:
    #     print("Edge:", edge.fact)

    if not results:
        return "No relevant facts found in database."

    context = ""
    for node in results.nodes:
      context += node.summary + "\n"
    for edge in results.edges:
      context += edge.fact + "\n"

    return context


final_template = """You are an expert Temporal Knowledge Graph Analyst.
Your job is to answer the user's question based STRICTLY on the provided "FACT FACTS".

### REAL FACTS FROM DATABASE:
{context}

### USER QUESTION:
{question}

### ANSWER:"""

final_prompt = ChatPromptTemplate.from_template(final_template)

full_chain = (
    {

        "context": hyde_chain | RunnableLambda(retrieve_strict_raw_edge),
        "question": RunnablePassthrough()
    }
    | final_prompt
    | llm
    | StrOutputParser()
)


<h1>Test Query</h1>

In [18]:

# --- 5. CHẠY THỬ ---
async def main():
    user_q = "Did John have anything with him when he was at city mall last night?"

    print(f"User hỏi: {user_q}")
    print("-" * 30)

    # Gọi chain
    response = await full_chain.ainvoke(user_q)

    print("-" * 30)
    print("🤖 AI Trả lời cuối cùng:")
    print(response)

await main()

User hỏi: Did John have anything with him when he was at city mall last night?
------------------------------
🔎 Đang query Graphiti (HyDE: Yes, John had his wallet and a shopping bag with him when he was at the city mall last night.
Node: City Mall is a location where John stated he was present near the food court yesterday evening.
Node: John was at the City Mall near the food court yesterday evening from 6:30 p.m. to 8:00 p.m., having dinner and meeting a friend.
Node: The food court is a location at the City Mall where John mentioned being present yesterday evening.
Node: Officer questioned John about his whereabouts on a specific evening; John stated he was at the City Mall near the food court from 6:30 p.m. to 8:00 p.m., having dinner and meeting a friend.
Node: John met a friend for dinner at the City Mall food court on the evening of the incident, arriving at 6:30 p.m. and leaving around 8:00 p.m.
------------------------------
🤖 AI Trả lời cuối cùng:
Based on the provided facts